In [1]:
import pandas as pd
import endpoints
from datetime import datetime, timedelta
# from config import SITE_EMAIL, SITE_PASSWORD
import time
import numpy as np
import requests
import json
from urllib.parse import quote
import os

In [2]:
today = datetime.today()
yesterday = today - timedelta(days=1)
today_str = today.strftime('%Y-%m-%d')
yesterday_str = yesterday.strftime('%Y-%m-%d')

In [3]:
SITE_EMAIL = os.getenv("SITE_EMAIL")
SITE_PASSWORD = os.getenv("SITE_PASSWORD")

In [4]:
cookies_dict = endpoints.get_cookies_local(SITE_EMAIL, SITE_PASSWORD)
session = endpoints.get_session()


In [63]:
def get_season_ratio(subject_id: int, today_str: str, session: requests.Session, cookies_dict: dict):
    session.headers.update({
        "Content-Type": "application/json",
        "Accept": "application/json",
        "User-Agent": "Mozilla/5.0 (compatible; DataCollector/1.0)"
    })

    url = f"https://eggheads.solutions/clientAnalytics/subject/seasonRatio/{subject_id}?dns-cache={today_str}_09-1"
    response = session.get(url, cookies=cookies_dict)

    if response.status_code == 200:
        data = response.json()
        return data.get('coefficients', {})
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        print(url)
        return {}

In [59]:
data = requests.get('https://eggheads.solutions/clientAnalytics/subject/seasonRatio/390?dns-cache=2025-09-26_12-0', cookies=cookies_dict)

In [62]:
data.text

'{"status":"ok","coefficients":[{"date":"2025-01-01T00:00:00+00:00","coefficient":1.3311788},{"date":"2025-01-02T00:00:00+00:00","coefficient":1.3964645},{"date":"2025-01-03T00:00:00+00:00","coefficient":1.2228465},{"date":"2025-01-04T00:00:00+00:00","coefficient":1.2082398},{"date":"2025-01-05T00:00:00+00:00","coefficient":1.1693792},{"date":"2025-01-06T00:00:00+00:00","coefficient":1.1320368},{"date":"2025-01-07T00:00:00+00:00","coefficient":1.1510634},{"date":"2025-01-08T00:00:00+00:00","coefficient":1.243558},{"date":"2025-01-09T00:00:00+00:00","coefficient":1.1727281},{"date":"2025-01-10T00:00:00+00:00","coefficient":1.1503732},{"date":"2025-01-11T00:00:00+00:00","coefficient":1.1227227},{"date":"2025-01-12T00:00:00+00:00","coefficient":1.128536},{"date":"2025-01-13T00:00:00+00:00","coefficient":1.4170213},{"date":"2025-01-14T00:00:00+00:00","coefficient":1.2789707},{"date":"2025-01-15T00:00:00+00:00","coefficient":1.1928551},{"date":"2025-01-16T00:00:00+00:00","coefficient":1.188

In [ ]:
subject_id = 390

In [ ]:
data = get_season_ratio(subject_id, today_str, session, cookies_dict)


In [66]:
df = pd.json_normalize(data)

In [68]:
df['subject_id'] = subject_id

,date,coefficient
0,2025-01-01T00:00:00+00:00,1.331179
1,2025-01-02T00:00:00+00:00,1.396464
2,2025-01-03T00:00:00+00:00,1.222846
3,2025-01-04T00:00:00+00:00,1.208240
4,2025-01-05T00:00:00+00:00,1.169379
...,...,...
361,2025-12-27T00:00:00+00:00,1.197803
362,2025-12-28T00:00:00+00:00,1.227122
363,2025-12-29T00:00:00+00:00,1.179053
364,2025-12-30T00:00:00+00:00,1.041077
